### Apparent Position of Mars in Skyfield

In [ ]:
# Apparent observe mars from earth with Skyfield
obs_mars_app_sf = earth_sf.at(obstime_mars_sf).observe(mars_sf).apparent()

# Build Skyfield angle arrays (RA, DEC) and distance array (delta)
ra_mars_app_sf_aa, dec_mars_app_sf_aa, delta_mars_app_sf_da = obs_mars_app_sf.radec(epoch='date')

# Extract degrees and AU to get arrays of astropy angles and distances
ra_mars_app_sf = ra_mars_app_sf_aa._degrees * deg
dec_mars_app_sf = dec_mars_app_sf_aa._degrees * deg
delta_mars_app_sf = delta_mars_app_sf_da.au * au

# Apparent observation of Mars from JPL
ra_mars_app_jpl = df_obs_mars.RA_apparent.values * deg
dec_mars_app_jpl = df_obs_mars.DEC_apparent.values * deg

In [ ]:
diff_app_sf = radec_diff('Skyfield astrometric', 'Skyfield apparent', 
                         ra1=ra_mars_sf, dec1=dec_mars_sf, 
                         ra2=ra_mars_app_sf, dec2=dec_mars_app_sf, 
                         obstime_mjd=obstime_mars_mjd, verbose=True)

**Conclusion:**<br>
Skyfield apparent() is different from astrometric by very similar amounts to JPL.<br>
This is encouraging.

In [ ]:
# Compute difference in angles
diff_app_sf = radec_diff('JPL apparent', 'Skyfield apparent', 
                         ra1=ra_mars_app_jpl, dec1=dec_mars_app_jpl, 
                         ra2=ra_mars_app_sf, dec2=dec_mars_app_sf, 
                         obstime_mjd=obstime_mars_mjd, verbose=True)

In [ ]:
def radec_app2dir(ra: float, dec: float, obstime_mjd: float) -> np.array:
    """
    Convert an apparent RA and DEC as of observation time to a unit displacement 
    vector u = (ux, uy, uz) in ecliptic plane.
    INPUTS:
        ra: An apparent Right Ascension in the ICRF; passed with units (default degrees)
        dec: An apparent Declination in the ICRF;  passed with units (default degrees)
        obtime_mjd: The observation time as a modified julian day
    RETURNS:
        u: An array [ux, uy, uz] on the unit sphere in the the ecliptic frame
    EXAMPLE:
        u = radec_app2dir(ra=76.391533*deg, dec=23.90881*deg, mjd=58600.0)
        (this is Mars viewed from Earth at mjd 58600 / 2019-04-27 with JPL RA and DEC)
    """
    # Build the observation as a SkyCoord in the ICRS (oriented with earth, origin at barycenter)
    obstime = astropy.time.Time(obstime_mjd, format='mjd')
    # obs_gcrs = SkyCoord(ra=ra, dec=dec, obstime=obstime, frame=astropy.coordinates.GCRS)
    # obs_gcrs = SkyCoord(lat=ra, lon=dec, obstime=obstime, frame=astropy.coordinates.GeocentricMeanEcliptic)
    # Convert to the barycentric ecliptic frame (oriented with ecliptic, origin at barycenter)
    obs_ecl = obs_gcrs.transform_to(BarycentricMeanEcliptic)
    u = obs_ecl.cartesian.xyz
    return u.value

In [ ]:
df_obs_mars[df_obs_mars.mjd==58600]

In [ ]:
# Build the observation as a SkyCoord
obstime = astropy.time.Time(58600.0, format='mjd')
# equinox = astropy.time.Time(2451545.0, format='jd')
equinox = obstime

# earth and body in earth's frame
frame_earth = astropy.coordinates.PrecessedGeocentric
# earth_geo = SkyCoord(ra=0*deg, dec=0*deg, distance=0*au, obstime=obstime, equinox=equinox, frame=frame_earth)
# body_geo = SkyCoord(ra=76.391533*deg, dec=23.90881*deg, distance=1*au, obstime=obstime, equinox=equinox, frame=frame_earth)
# earth_geo = SkyCoord(lat=0*deg, lon=0*deg, distance=0*au, obstime=obstime, equinox=equinox, frame=frame_earth)
# body_geo = SkyCoord(lat=76.391533*deg, lon=23.90881*deg, distance=1*au, obstime=equinox, equinox=obstime, frame=frame_earth)

earth_geo = SkyCoord(ra=0*deg, dec=0*deg, distance=0*au, obstime=obstime, equinox=equinox, frame=frame_earth)
body_geo = SkyCoord(ra=76.391533*deg, dec=23.90881*deg, distance=1*au, obstime=obstime, equinox=equinox, frame=frame_earth)

# earth and body in BME
earth_bme = earth_geo.transform_to(BarycentricMeanEcliptic)
body_bme = body_geo.transform_to(BarycentricMeanEcliptic)

# displacement
q_rel = body_bme.cartesian.xyz - earth_bme.cartesian.xyz
u = q_rel.value / np.linalg.norm(q_rel)
u

In [ ]:
u_mars

In [ ]:
err = u - u_mars
np.rad2deg(np.linalg.norm(err))*3600

In [ ]:
u_mars = np.array([0.21954, 0.97543, 0.018417])
u1 = radec2dir(ra=76.107414*deg, dec=23.884883*deg, obstime_mjd=58600.0)
err1 = np.rad2deg(np.linalg.norm(u1 - u_mars))*3600

u2 = radec_app2dir(ra=76.391533*deg, dec=23.90881*deg, obstime_mjd=58600.0)
err2 = np.rad2deg(np.linalg.norm(u2 - u_mars))*3600

print(f'Direction of Mars error')
print(f'astrometric RA/DEC: {err1:8.3f} arc seconds')
print(f'apparent    RA/DEC: {err2:8.3f} arc seconds')